# 10 - The Guardian

## Qualunque anno

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import re
import numpy as np
import pandas as pd
from requests import get
from pandas import read_csv
from unidecode import unidecode
from bs4 import BeautifulSoup as BS

In [2]:
df = read_csv('tables/nome_nomechiave.csv', encoding='utf8')
s1 = 'https://www.google.com/search?q='
s2 = '+site:www.theguardian.com&source=lnms&tbm=nws'
df['urls_all'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['guardian_google_all'] = np.nan

In [ ]:
for i in range(300):
    if isinstance(df.guardian_google_all[i], float):
        url = df.urls_all[i]
        result = BS(get(url).text, 'lxml').select('#resultStats')[0].text
        df.loc[i, 'guardian_google_all'] = result
        #print((i, df.nome[i]), end=' ')

## 2018

In [7]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:www.theguardian.com&tbm=nws&source=lnt&tbs=qdr:y'
df['url_18'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['guardian_google_18'] = np.nan

In [ ]:
driver = webdriver.Chrome(executable_path='chromedriver')
driver.maximize_window()

results = []

for i in range(300):
    if isinstance(df.guardian_google_18[i], float):
        url = df.url_18[i]
        driver.get(url)
        sleep(np.random.uniform(2))
        el = driver.find_element_by_link_text('Strumenti')
        webdriver.ActionChains(driver).move_to_element(el).click(el).perform()
        sleep(np.random.uniform(2))
        result = driver.find_element_by_id('resultStats').text
        df.loc[i, 'guardian_google_18'] = result
        #print((i,df.nome[i]), end=' ')
    
driver.quit()

## Pulizia dati

In [50]:
df[['guardian_google_all', 'guardian_google_18']].head()

,guardian_google_all,guardian_google_18
0,Circa 639 risultati,"Circa 354 risultati (0,22 secondi)"
1,Circa 3.300 risultati,"Circa 687 risultati (0,16 secondi)"
2,Circa 5.860 risultati,"Circa 964 risultati (0,19 secondi)"
3,Circa 1.960 risultati,"Circa 852 risultati (0,20 secondi)"
4,Circa 4.420 risultati,"Circa 1.000 risultati (0,21 secondi)"


In [ ]:
def f(x):
    x=x.replace('.','')
    x=re.findall('(\d+)',x)
    if x:
        return x[0]

df2 = df[['guardian_google_all', 'guardian_google_18']]

df2.loc[0:300, 'guardian_google_all'] = df2.guardian_google_all.apply(f)
df2.loc[0:300, 'guardian_google_18'] = df2.guardian_google_18.apply(f)

In [49]:
df2.head()

,guardian_google_all,guardian_google_18
0,639,354
1,3300,687
2,5860,964
3,1960,852
4,4420,1000


In [51]:
df2.to_csv('columns/guardian_data.csv')